In [ ]:

import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
from dateutil.relativedelta import relativedelta
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
API_KEY = 'wHSk4qAJFso7oBI9YAT7DrkDNunyU1T8'

In [ ]:
MAX_RETRIES = 5
INITIAL_DELAY = 1

def send_request(date):
    base_url = 'https://api.nytimes.com/svc/news/v3/content/nyt/business/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + API_KEY
    retries = 0
    while retries < MAX_RETRIES:
        try:
            response = requests.get(url, verify=False).json()
            return response
        except Exception as e:
            print(f"Request failed: {e}")
            retries += 1
            delay = INITIAL_DELAY * (2 ** retries)
            print(f"Retrying in {delay} seconds...")
            time.sleep(delay)
    return None


def is_valid(article, date):
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline


def parse_response(response):
    data = {'headline': [],
        'date': [],
        'web_url': [],
        'doc_type': [],
        'lead_paragraph': [],
        'material_type': [],
        'author': [],
        'section': [],
        'subsection': [],
        'keywords': []}

    articles = response['response']['docs']
    for article in articles:
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main'])
            if 'section_name' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            if 'lead_paragraph' in article:
                data['lead_paragraph'].append(article['lead_paragraph'])
            else:
                data['lead_paragraph'].append(None)
            if 'web_url' in article:
                data['web_url'].append(article['web_url'])
            else:
                data['web_url'].append(None)
            if 'subsection_name' in article:
                data['subsection'].append(article['subsection_name'])
            else:
                data['subsection'].append(None)
            if 'byline' in article:
                data['author'].append(article['byline']['original'])
            else:
                data['author'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article:
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data)


def get_data(dates):
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        print('Working on ' + str(date) + '...')
        csv_path = 'headlines/' + date[0] + '-' + date[1] + '.csv'
        if not os.path.exists(csv_path): # If we don't already have this month
            response = send_request(date)
            if response is not None:
                df = parse_response(response)
                total += len(df)
                df.to_csv(csv_path, index=False)
                print('Saving ' + csv_path + '...')
    print('Number of articles collected: ' + str(total))

In [ ]:

end = datetime.date(2022, 9, 29)
start = datetime.date(2021, 9, 30)

In [ ]:
months = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]


In [ ]:
get_data(months)

In [ ]:
import os
import glob
import pandas as pd
os.chdir("/content/headlines") ## use Google Colab

In [ ]:

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:

#combine in a single file
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv2.csv", index=False, encoding='utf-8-sig')